In [ ]:
# Minimal test notebook for the AI agent
# - INI reading (small subset)
# - File import (tab-delimited, similar to pipeline)
# - AI placeholder cells (to implement llm_transform and prompt templates)
# - Export example

import configparser
import ast
from pathlib import Path


def _parse_list_option(val):
    if not val:
        return []
    try:
        parsed = ast.literal_eval(val)
        if isinstance(parsed, (list, tuple)):
            return [str(x).strip() for x in parsed]
    except Exception:
        return [c.strip() for c in str(val).split(',') if c.strip()]
    return []


def find_config(name="lang_pipeline.ini"):
    p = Path.cwd()
    for _ in range(6):
        candidate = p / name
        if candidate.exists():
            return candidate
        p = p.parent
    return next(Path.cwd().rglob(name), None)


cfg_path = find_config()
CONFIG = {}
if cfg_path:
    cp = configparser.ConfigParser()
    cp.read(cfg_path)
    if 'pipeline' in cp:
        sec = cp['pipeline']
        CONFIG['source'] = sec.get('source', None)
        CONFIG['source_browser'] = sec.getboolean('source_browser', fallback=True)
        CONFIG['preview'] = sec.getboolean('preview', fallback=True)
        CONFIG['preview_rows'] = sec.getint('preview_rows', fallback=20)
        CONFIG['columns_export_filter'] = sec.getboolean('columns_export_filter', fallback=False)
        CONFIG['columns_export'] = _parse_list_option(sec.get('columns_export', ''))
        # agent placeholders
        CONFIG['ai_provider'] = sec.get('ai_provider', '')
        CONFIG['ai_model'] = sec.get('ai_model', '')
    print(f"Loaded config from: \n{cfg_path}")
else:
    print("No lang_pipeline.ini found; using defaults")


In [ ]:
# File selection (interactive or from INI)
import platform, subprocess
from pathlib import Path

if not CONFIG.get('source_browser', True) and CONFIG.get('source'):
    input_path = Path(CONFIG['source'])
    if not input_path.exists():
        raise FileNotFoundError(f"Configured (INI) source not found: {input_path}")
    print("Configured (INI) source used:", input_path)
else:
    try:
        from tools.ui_native import pick_file
    except Exception:
        def pick_file(filter_str=None):
            system = platform.system()
            if system == "Darwin":
                script = '''
                set theFile to choose file
                POSIX path of theFile
                '''
                res = subprocess.run(["osascript", "-e", script], capture_output=True, text=True)
                return res.stdout.strip()
            elif system == "Windows":
                ps_script = r'''
                Add-Type -AssemblyName System.Windows.Forms
                $ofd = New-Object System.Windows.Forms.OpenFileDialog
                $ofd.Filter = "All files (*.*)|*.*"
                if ($ofd.ShowDialog() -eq "OK") { Write-Output $ofd.FileName }
                '''
                res = subprocess.run(["powershell", "-NoProfile", "-Command", ps_script], capture_output=True, text=True)
                return res.stdout.strip()
            else:
                raise NotImplementedError("No native file dialog for this OS")

    pick_path = pick_file("CSV files (*.csv)|*.csv")
    if not pick_path:
        raise FileNotFoundError("No file selected")

    input_path = Path(pick_path)
    print(f"Selected input file: \n{input_path}")

In [ ]:
# Line-by-line import (tab-delimited)
import csv

rows = []
bad_rows = []
total_lines = 0
with input_path.open("r", encoding="utf-8", errors="replace") as f:
    reader = csv.reader(f, delimiter="\t", quotechar='"', escapechar='\\')
    try:
        header = next(reader)
    except StopIteration:
        header = []
    for line_number, line in enumerate(reader, start=2):
        total_lines += 1
        if len(line) == len(header):
            rows.append(dict(zip(header, line)))
        elif len(line) > len(header) and len(line) % len(header) == 0:
            for i in range(0, len(line), len(header)):
                subline = line[i:i+len(header)]
                rows.append(dict(zip(header, subline)))
        else:
            bad_rows.append((line_number, line))

expected_columns = header if 'header' in locals() else []
print(f"Header: \n{expected_columns}\n")
print(f"Imported {len(rows)} rows ({total_lines} lines read).")
if bad_rows:
    print(f"{len(bad_rows)} malformed lines found.")
else:
    print("No malformed lines found.")

In [ ]:
# AI adapter placeholder
# This is where we'll implement the llm_transform function in the real agent.
# Keep it simple for tests: pass-through or a mocked transformation.

def llm_transform(records):
    """Placeholder LLM transform for testing.
    Input: list[dict]
    Output: list[dict]
    For now, copy 'translation' -> 'translation_ai' to simulate work.
    """
    out = []
    for r in records:
        nr = dict(r)
        # simulate AI adding a new column
        nr['translation_ai'] = nr.get('translation', '')
        nr['ai_notes'] = ''
        out.append(nr)
    return out

# Run a quick transform on the imported rows (for interactive testing)
processed_rows = llm_transform(rows)
print(f"Processed {len(processed_rows)} rows via llm_transform (placeholder).")

In [ ]:
# Export result (tab-delimited)
import csv

# Choose export columns: keep original columns + ai columns added by llm_transform
export_fieldnames = list(expected_columns) + [c for c in ("translation_ai","ai_notes") if c not in expected_columns]

output_suffix = "_agent_test_out"
stem = input_path.stem
ext = input_path.suffix
output_path = input_path.with_name(f"{stem}{output_suffix}{ext}")
print(f"Exporting to: {output_path}")

with output_path.open("w", encoding="utf-8", newline="") as f:
    writer = csv.DictWriter(f, fieldnames=export_fieldnames, delimiter="\t", extrasaction='ignore', restval='')
    writer.writeheader()
    writer.writerows(processed_rows)

print("Export complete")